## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

d:\project at home\project-plan b\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [11]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32, 3, 3,activation='relu',input_shape=(32,32,3)))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(BatchNormalization(axis=-1, momentum=0.8, epsilon=0.01))

'''自己決定MaxPooling2D放在哪裡'''
# classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(32, 3, 3, activation='relu'))
classifier.add(BatchNormalization(axis=-1, momentum=0.8, epsilon=0.03))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(128, activation='relu')) 
#output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='sigmoid'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=20)

d:\project at home\project-plan b\venv\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", input_shape=(32, 32, 3...)`
  after removing the cwd from sys.path.
d:\project at home\project-plan b\venv\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  if sys.path[0] == '':
d:\project at home\project-plan b\venv\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=10)`


Epoch 1/20
50000/50000 [==============================] - 37s 743us/step - loss: 1.4005 - accuracy: 0.5185
Epoch 2/20
50000/50000 [==============================] - 37s 739us/step - loss: 0.8598 - accuracy: 0.6930
Epoch 3/20
50000/50000 [==============================] - 37s 748us/step - loss: 0.6021 - accuracy: 0.7843
Epoch 4/20
50000/50000 [==============================] - 37s 737us/step - loss: 0.4114 - accuracy: 0.8505
Epoch 5/20
50000/50000 [==============================] - 37s 737us/step - loss: 0.2708 - accuracy: 0.9040
Epoch 6/20
50000/50000 [==============================] - 38s 761us/step - loss: 0.1913 - accuracy: 0.9321
Epoch 7/20
50000/50000 [==============================] - 37s 733us/step - loss: 0.1323 - accuracy: 0.9547
Epoch 8/20
50000/50000 [==============================] - 37s 734us/step - loss: 0.1195 - accuracy: 0.9579
Epoch 9/20
50000/50000 [==============================] - 37s 737us/step - loss: 0.1125 - accuracy: 0.9607
Epoch 10/20
50000/50000 [============

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [10]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[1.3455086e-03, 2.9456403e-11, 5.0924493e-05, 6.9430310e-01,
        2.1966207e-03, 1.1868378e-07, 2.8188884e-07, 3.0138224e-06,
        5.8127195e-04, 3.4607449e-11]], dtype=float32)